In [ ]:
!pip install -q -U google-generativeai

In [ ]:
# Necessary packages
import pathlib
import textwrap
import json

import google.generativeai as genai
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from IPython.display import display
from IPython.display import Markdown
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from google.colab import userdata

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [ ]:
# Muat file Excel
df = pd.read_excel('/content/Sentimen_3.xlsx')

In [ ]:
df.head(2)

,Sentimen,True_Label
0,Sangat mendukung karena sangat bermanfaat bagi...,2
1,Sangat mendukung karena untuk kenyamanan wisat...,2


In [ ]:
df['pred_label'] = ''

In [ ]:
df.head(2)

,Sentimen,True_Label,pred_label
0,Sangat mendukung karena sangat bermanfaat bagi...,2,
1,Sangat mendukung karena untuk kenyamanan wisat...,2,


In [ ]:
# Convert the DataFrame to JSON using the to_json() method

json_data = df[['Sentimen','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"Sentimen":"Sangat mendukung karena sangat bermanfaat bagi wisatawan muslim dan akan menjadi wisata edukasi bagi anak-anak","pred_label":""},{"Sentimen":"Sangat mendukung karena untuk kenyamanan wisata muslim sesuai syariat agama","pred_label":""},{"Sentimen":"Sangat mendukung karena wisatawan tidak dibatasi untuk melaksanakan kewajiban beribadah","pred_label":""},{"Sentimen":"Sangat mendukung karna disaat lokasi wisatanya muslim friendly maka bagi non muslim pun tetap nyaman dan aman berwisata di lokasi tersebut sehingga menciptakan lokasi wisata yang sama2 nyaman bagi siapapun","pred_label":""},{"Sentimen":"Sangat mendukung sebab orang berwisata untuk memperoleh rasa bahagia dan nyaman setelah penat terikat rutinitas kehidupan Dan yang di butuhkan bukan hanya pemuas phisyk saja tapi emotional serta spiritualnya Jadi objek wisata harus lengkap menyediakan sarana dan prasarana yang mampu memenuhi kebutuhan dan memberikan kepuasan para wisatawan","pred_label":""},{"Sentimen":"Sangat m

In [ ]:
prompt = f"""
Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
{json_data}
```
"""

print(prompt)


Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
[{"Sentimen":"Sangat mendukung karena sangat bermanfaat bagi wisatawan muslim dan akan menjadi wisata edukasi bagi anak-anak","pred_label":""},{"Sentimen":"Sangat mendukung karena untuk kenyamanan wisata muslim sesuai syariat agama","pred_label":""},{"Sentimen":"Sangat mendukung karena wisatawan tidak dibatasi untuk melaksanakan kewajiban beribadah","pred_label":""},{"Sentimen":"Sangat mendukung karna disaat lokasi wisatanya muslim friendly m

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
[{"Sentimen":"Sangat mendukung karena sangat bermanfaat bagi wisatawan muslim dan akan menjadi wisata edukasi bagi anak-anak","pred_label":2},{"Sentimen":"Sangat mendukung karena untuk kenyamanan wisata muslim sesuai syariat agama","pred_label":2},{"Sentimen":"Sangat mendukung karena wisatawan tidak dibatasi untuk melaksanakan kewajiban beribadah","pred_label":2},{"Sentimen":"Sangat mendukung karna disaat lokasi wisatanya muslim friendly maka bagi non muslim pun tetap nyaman dan aman berwisata di lokasi tersebut sehingga menciptakan lokasi wisata yang sama2 nyaman bagi siapapun","pred_label":2},{"Sentimen":"Sangat mendukung sebab orang berwisata untuk memperoleh rasa bahagia dan nyaman setelah penat terikat rutinitas kehidupan Dan yang di butuhkan bukan hanya pemuas phisyk saja tapi emotional serta spiritualnya Jadi objek wisata harus lengkap menyediakan sarana dan prasarana yang mampu memenuhi kebutuhan dan memberikan kepuasan para wisatawan","pred_label":2},{"Sentimen":"Sangat me

In [ ]:
# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,Sentimen,pred_label
0,Sangat mendukung karena sangat bermanfaat bagi...,2
1,Sangat mendukung karena untuk kenyamanan wisat...,2
2,Sangat mendukung karena wisatawan tidak dibata...,2
3,Sangat mendukung karna disaat lokasi wisatanya...,2
4,Sangat mendukung sebab orang berwisata untuk m...,2
5,Sangat mendukung sehingga memberikan rasa nyam...,2
6,Sangat mendukung sekali apalagi hal kreatif da...,2
7,Sangat mendukung sekali d adakan nya kebijakan...,2
8,Sangat mendukung sekali karena mempermudah kit...,2
9,Sangat mendukung terutama untuk area yang isla...,2


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

df['pred_label'] = df_sample['pred_label'].values
df

,Sentimen,True_Label,pred_label
0,Sangat mendukung karena sangat bermanfaat bagi...,2,2
1,Sangat mendukung karena untuk kenyamanan wisat...,2,2
2,Sangat mendukung karena wisatawan tidak dibata...,2,2
3,Sangat mendukung karna disaat lokasi wisatanya...,2,2
4,Sangat mendukung sebab orang berwisata untuk m...,2,2
5,Sangat mendukung sehingga memberikan rasa nyam...,2,2
6,Sangat mendukung sekali apalagi hal kreatif da...,2,2
7,Sangat mendukung sekali d adakan nya kebijakan...,2,2
8,Sangat mendukung sekali karena mempermudah kit...,2,2
9,Sangat mendukung terutama untuk area yang isla...,2,2


In [ ]:
# prompt: Menggunakan df DataFrame: saya ingin menyimpan file kedalam bentuk excel

df.to_excel('data 3.xlsx')